# 痞客邦資料分析

## Preparation

In [15]:
# module import
import re
import json
from bs4 import BeautifulSoup
from pprint import pprint

In [90]:
# the file path of pixnet
data_file_path = "./2017-pixnet-hackathon-data/article/food.json"

# the file path to our data
our_data_file_path = "./data.json"

## Load the json file

In [17]:
data = []
with open(data_file_path) as data_file:
    for line in data_file:
        data.append(json.loads(line))

In [18]:
print "Number of blogs: {}".format(len(data))

Number of blogs: 9635


In [19]:
# filter out data without content
data = [d for d in data if 'content' in d]


## Parse the phone

In [45]:
#phone_pattern = r"電話?([0-9\-])"
phone_pattern = r"電話.{,10}[0-9\-\(\)\s]+"
phone_format = r"([0-9]+)[^0-9]*([0-9]*)[^0-9]*([0-9]*)[^0-9]*([0-9]*)"
debug_pattern = "電話.{50}"

In [ ]:
# pattern refine process
for d in data:
    # remove the html tag
    content_without_html = BeautifulSoup(d['content']).text
    
    if re.findall(phone_pattern, content_without_html.encode('utf-8')):
        for raw_phone in re.findall(phone_pattern, content_without_html.encode('utf-8')):
            p = re.findall(phone_format, raw_phone)
            #print p
            #print raw_phone

## Parse the address

In [81]:
address_pattern1 = r"地址.{,200}"
address_pattern2 = r"地址：*\s*(.*(號|樓)\s*\)*)"

In [85]:
def get_address(json_blog):
    result = []
    s_contain_addr = re.findall(address_pattern1, json_blog['content'].encode('utf-8'))
    for sca in s_contain_addr:
        s_without_html = BeautifulSoup(sca).text
        s_addr = re.findall(address_pattern2, s_without_html.encode('utf-8'))
        for sa in s_addr:
            result.append(sa[0])
    return result

In [89]:
count = 0
# pattern refine process
for d in data:
    addresses = get_address(d)
    if addresses:
        count += 1
        #print addresses[0]

In [87]:
count

4399

## Write json

In [88]:
our_data = {}
for d in data:
    id = d['article_id']
    our_data[id] = {}
    our_data[id]['url'] = d['url']
    our_data[id]['hits'] = d['hits']
    
    # get address
    address = get_address(d)
    
    if address:
        our_data[id]['address'] = address
        our_data[id]['usable'] = True
    else:
        our_data[id]['usable'] = False

In [92]:
with open(our_data_file_path, 'w') as data_file:
    json.dump(our_data, data_file, ensure_ascii=False)